# Get alert history for all devices

In [1]:
import os
import sys
from mxnet.gluon import data as gdata

def load_data_alerts(batch_size, root=os.path.join('~', '.mxnet', 'datasets', 'alerts')):
    root = os.path.expanduser(root)
    transformer = []
        
    transformer += [gdata.vision.transforms.ToTensor()]
    transformer = gdata.vision.transforms.Compose(transformer)

    mnist_train = gdata.vision.FashionMNIST(root=root, train=True)
    mnist_test = gdata.vision.FashionMNIST(root=root, train=False)
    num_workers = 0 if sys.platform.startswith('win32') else 4
    
    train_iter = gdata.DataLoader(mnist_train.transform_first(transformer),
                                  batch_size, shuffle=True,
                                  num_workers=num_workers)
    
    test_iter = gdata.DataLoader(mnist_test.transform_first(transformer),
                                 batch_size, shuffle=False,
                                 num_workers=num_workers)
    
    return train_iter, test_iter

In [2]:
from mxnet import autograd, nd

num_inputs = 784
num_outputs = 10
batch_size = 256

train_iter, test_iter = load_data_alerts(batch_size)

W = nd.random.normal(scale=0.01, shape=(num_inputs, num_outputs))
b = nd.zeros(num_outputs)
params = [W, b]

for param in params:
    param.attach_grad()

In [3]:
def softmax(X):
    X_exp = nd.exp(X)
    partition = X_exp.sum(axis=1, keepdims=True)
    return X_exp / partition

In [4]:
def net(X):
    return softmax(nd.dot(X.reshape(-1, num_inputs), W) + b)

In [5]:
def cross_entropy(y_hat, y):
    return - nd.pick(y_hat, y).log()

In [6]:
def accuracy(y_hat, y):
    return (y_hat.argmax(axis=1) == y.astype('float32')).mean().asscalar()

In [7]:
def evaluate_accuracy(data_iter, net):
    acc = 0.
    for X, y in data_iter:
        acc += accuracy(net(X), y)
    return acc / len(data_iter)

In [8]:
evaluate_accuracy(test_iter, net)

0.09453125

In [9]:
def SGD(params, lr):
    for param in params:
        param[:] = param - lr * param.grad

In [10]:
from mxnet import autograd

num_epochs, learning_rate = 5, 0.1

for epoch in range(num_epochs):
    train_loss = 0.
    train_acc = 0.
    for X, y in train_iter:
        with autograd.record():
            y_hat = net(X)
            loss = cross_entropy(y_hat, y)
        loss.backward()
        
        SGD(params, learning_rate / batch_size)
        
        train_loss += loss.mean().asscalar()
        train_acc += accuracy(y_hat, y)
        
    test_acc = evaluate_accuracy(test_iter, net)

    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch + 1, train_loss/len(train_iter), train_acc/len(train_iter), test_acc))


Epoch 1. Loss: 0.802150, Train acc 0.738691, Test acc 0.798047
Epoch 2. Loss: 0.585872, Train acc 0.806527, Test acc 0.815820
Epoch 3. Loss: 0.556710, Train acc 0.814268, Test acc 0.817871
Epoch 4. Loss: 0.524453, Train acc 0.822645, Test acc 0.829395
Epoch 5. Loss: 0.523110, Train acc 0.824346, Test acc 0.835254


In [11]:
params

[
 [[ 0.02208608  0.00773389  0.01038804 ... -0.0045156   0.00577459
   -0.01856295]
  [-0.01968029 -0.00208902  0.00238516 ...  0.01465095  0.00689538
    0.00358461]
  [ 0.01145082  0.00107873 -0.010196   ... -0.01077379 -0.01143824
   -0.01332576]
  ...
  [-0.01379873 -0.0178848   0.04781896 ... -0.00039847 -0.01541492
   -0.00778142]
  [-0.00794938  0.01271872 -0.00818972 ... -0.02308376 -0.02098199
    0.0022121 ]
  [-0.01107594  0.00097882  0.00648582 ...  0.0037565  -0.00105099
    0.00027636]]
 <NDArray 784x10 @cpu(0)>, 
 [ 0.12101663 -0.15155849 -0.09033381  0.0780093  -0.62617314  1.4327859
   0.3022117  -0.11178205 -0.31675348 -0.63742304]
 <NDArray 10 @cpu(0)>]